In [217]:
from pathlib import Path

import pandas as pd

In [218]:
pd.options.display.float_format = "{:_.4f}".format

In [219]:
DADOS: Path = Path(".").parent.resolve() / "data" / "raw"
AMOSTRAS: Path = Path(".").parent.resolve() / "data" / "sample"

# Carga dos dados

In [220]:
names_map = {
    "Luiz Inácio Lula Da Silva": "Lula",
    "Jose Maria Eymael": "Eymael",
    "Constituinte Eymael": "Eymael",
    "Kelmon Luis Da Silva Souza": "Padre Kelmon",
    "Brancos": "Branco",
    "Voto Branco": "Branco",
    "Luiz Felipe Chaves D Avila": "Felipe D'Avila",
    "Soraia Thronicke": "Soraya Thronicke",
    "Soraya Vieira Thronicke": "Soraya Thronicke",
    "Ciro Ferreira Gomes": "Ciro Gomes",
    "Sofia Padua Manzano": "Sofia Manzano",
    "Jair Messias Bolsonaro": "Jair Bolsonaro",
    "Simone Nassar Tebet": "Simone Tebet",
    "Leonardo Péricles Vieira Roque": "Léo Péricles",
    "Voto Nulo": "Nulo",
    "Nulos": "Nulo",
    "Vera Lucia Pereira Da Silva Salgado": "Vera Lucia",
    "Felipe D´Avila": "Felipe D'Avila",
}

In [230]:
dfs = []

for file in DADOS.glob("*.csv"):
    cargo, turno, *_ = file.stem.split("_")
    df = (
        pd.read_csv(file, dtype="category")
        .iloc[:, -2:]
        .rename(columns=str.casefold)
        .rename(columns=lambda name: "nome" if name != "estado" else name)
        .assign(
            nome=lambda df_: df_["nome"].str.title(),
            cargo=cargo,
            turno=int(turno)
        )
        .assign(
            nome=lambda df_: df_["nome"].apply(lambda s: names_map.get(s, s))
        )
    )
    dfs.append(df)

populacao = pd.concat(dfs).astype({
    "estado": "category", "cargo": "category"
})

# Geração das amostras

In [ ]:
sample_sizes = (
    pd.DataFrame([
        ['AC', 1, "Governador", 455_438, 2389],
        ['AC', 1, "Senador", 455_438, 2389],
        ['RO', 1, "Governador", 925_763, 2395],
        ['RO', 1, "Senador", 925_763, 2395],
        ['RO', 2, "Governador", 925563, 2395],
        ["AC", 1, "Presidente", 455_903, 110],
        ["RO", 1, "Presidente", 926_827, 224],
        ["AM", 1, "Presidente", 2_113_771, 512],
        ["RR", 1, "Presidente", 305_404, 74],
        ["AP", 1, "Presidente", 442_842, 107],
        ["PA", 1, "Presidente", 4_789_311, 1_159],
        ["TO", 1, "Presidente", 891_449, 216],
        ["AC", 2, "Presidente", 420_760, 104],
        ["RO", 2, "Presidente", 926_517, 230],
        ["AM", 2, "Presidente", 2_067_875, 513],
        ["RR", 2, "Presidente", 286_269, 71],
        ["AP", 2, "Presidente", 400_683, 99],
        ["PA", 2, "Presidente", 4_701_740, 1_167],
        ["TO", 2, "Presidente", 871_238, 216],

    ],
    columns=["estado", "turno", "cargo", "N", "n"])
    .set_index(["estado", "turno", "cargo"])
)
sample_sizes

N     n
estado turno cargo                    
AC     1     Governador   455438  2389
             Senador      455438  2389
RO     1     Governador   925763  2395
             Senador      925763  2395
       2     Governador   925563  2395
AC     1     Presidente   455903   110
RO     1     Presidente   926827   224
AM     1     Presidente  2113771   512
RR     1     Presidente   305404    74
AP     1     Presidente   442842   107
PA     1     Presidente  4789311  1159
TO     1     Presidente   891449   216
AC     2     Presidente   420760   104
RO     2     Presidente   926517   230
AM     2     Presidente  2067875   513
RR     2     Presidente   286269    71
AP     2     Presidente   400683    99
PA     2     Presidente  4701740  1167
TO     2     Presidente   871238   216

In [ ]:
random_state = 789

for (uf, turno, cargo), tamanho in sample_sizes.iterrows():
    sample = (
        populacao.query(
            f"estado == {uf!r} and turno == {turno} and cargo == {cargo!r}"
        )
        .sort_values(by="nome")
        .sample(tamanho.n, replace=False, random_state=random_state)
    )
    sample[["nome", "estado"]].to_csv(
        AMOSTRAS / f"{cargo}_{turno}_turno_{uf}.csv",
    )
    print(uf, turno, cargo, sample.shape[0])

AC 1 Governador 2389
AC 1 Senador 2389
RO 1 Governador 2395
RO 1 Senador 2395
RO 2 Governador 2395
AC 1 Presidente 110
RO 1 Presidente 224
AM 1 Presidente 512
RR 1 Presidente 74
AP 1 Presidente 107
PA 1 Presidente 1159
TO 1 Presidente 216
AC 2 Presidente 104
RO 2 Presidente 230
AM 2 Presidente 513
RR 2 Presidente 71
AP 2 Presidente 99
PA 2 Presidente 1167
TO 2 Presidente 216


In [ ]:
dfs = []

for file in AMOSTRAS.glob("*.csv"):
    cargo, turno, *_ = file.stem.split("_")
    df = (
        pd.read_csv(file, dtype="category")
        .iloc[:, -2:]
        .rename(columns=str.casefold)
        .rename(columns=lambda name: "nome" if name != "estado" else name)
        .assign(
            nome=lambda df_: df_["nome"].str.title(),
            cargo=cargo,
            turno=int(turno)
        )
        .assign(
            nome=lambda df_: df_["nome"].apply(lambda s: names_map.get(s, s))
        )
    )
    dfs.append(df)

amostra = (
    pd.concat(dfs)
    .astype({
        "estado": "category", "cargo": "category"
    })
)

# Proporções

## Amostrais

In [176]:
(
    amostra
    .query("cargo != 'Presidente'")
    .groupby(["cargo", "estado", "turno"])
    ["nome"].apply(lambda s: s.value_counts(normalize=True))
    .to_frame("proporcao")
)

/var/folders/xs/y768pf612tscp94by_62zm040000gn/T/ipykernel_11958/902614048.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["cargo", "estado", "turno"])


proporcao
cargo      estado turno                                
Governador AC     1     Gladson Cameli           0.5303
                        Jorge Viana              0.2265
                        Mara Rocha               0.1059
                        Petecão                  0.0611
                        Nulo                     0.0469
                        Branco                   0.0205
                        Marcio Bittar            0.0067
                        Professor Nilson         0.0013
                        David Hall               0.0008
           RO     1     Coronel Marcos Rocha     0.3587
                        Marcos Rogerio           0.3491
                        Léo Moraes               0.1203
                        Daniel Pereira           0.0848
                        Nulo                     0.0576
                        Branco                   0.0280
                        Pimenta De Rondonia      0.0017
                  2     Coronel Marcos Rocha     0.5106
                        Marcos Rogério           0.4397
                        Nulo                     0.0313
                        Branco                   0.0184
Senador    AC     1     Alan Rick                0.3365
                        Ney Amorim               0.1586
                        Dr. Jenilson Leite       0.1503
                        Nazaré Araújo            0.0921
                        Marcia Bittar            0.0892
                        Dra. Vanda Milani        0.0670
                        Nulo                     0.0607
                        Branco                   0.0398
                        Sanderson Moura          0.0042
                        Dimas Sandas             0.0017
           RO     1     Jaime Bagattoli          0.3094
                        Mariana Carvalho         0.2873
                        Jaqueline Cassol         0.1056
                        Expedito Junior          0.0944
                        Nulo                     0.0777
                        Acir Gurgacz             0.0664
                        Branco                   0.0480
                        Pastor Josinelio         0.0071
                        Dra Rosangela Lázaro     0.0042

In [202]:
proporcao_estrato = (
    amostra
    .query("cargo == 'Presidente'")
    .groupby(["cargo", "estado", "turno"])
    ["nome"]
    .apply(lambda s: s.value_counts(normalize=True))
    .to_frame("proporcao")
    .assign(
        nome=lambda df_: df_.index.get_level_values(-1)
    )
    .droplevel(-1)
)
proporcao_estrato

/var/folders/xs/y768pf612tscp94by_62zm040000gn/T/ipykernel_11958/1730697234.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["cargo", "estado", "turno"])


proporcao            nome
cargo      estado turno                           
Presidente AC     1         0.6818  Jair Bolsonaro
                  1         0.2545            Lula
                  1         0.0273      Ciro Gomes
                  1         0.0182          Branco
                  1         0.0091    Simone Tebet
...                            ...             ...
           TO     1         0.0139          Branco
                  2         0.5231            Lula
                  2         0.4306  Jair Bolsonaro
                  2         0.0370            Nulo
                  2         0.0093          Branco

[75 rows x 2 columns]

In [211]:
peso_estrato = (
    sample_sizes
    .query("cargo == 'Presidente'")
    .groupby("turno", as_index=False)
    ["N"].apply(lambda s: s / s.sum())
    .to_frame("Wh")
    .droplevel(0)
    .reorder_levels(["cargo", "estado", "turno"])
)

In [216]:
(
    proporcao_estrato
    .join(peso_estrato)
    .groupby(["nome", "turno"])
    .apply(lambda df_: (df_["proporcao"] * df_["Wh"]).sum())
    .unstack("turno")
)

/var/folders/xs/y768pf612tscp94by_62zm040000gn/T/ipykernel_11958/559333321.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df_: (df_["proporcao"] * df_["Wh"]).sum())


turno,1,2
nome,,
Branco,0.0100,0.0112
Ciro Gomes,0.0241,NaN
Felipe D'Avila,0.0025,NaN
Jair Bolsonaro,0.4497,0.4805
Lula,0.4512,0.4862
Nulo,0.0166,0.0221
Padre Kelmon,0.0012,NaN
Simone Tebet,0.0412,NaN
Soraya Thronicke,0.0033,NaN


## Populacionais

In [231]:
(
    populacao
    .query("cargo != 'Presidente'")
    .groupby(["cargo", "estado", "turno"])
    ["nome"].apply(lambda s: s.value_counts(normalize=True))
    .to_frame("proporcao")
)

/var/folders/xs/y768pf612tscp94by_62zm040000gn/T/ipykernel_11958/939325445.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["cargo", "estado", "turno"])


proporcao
cargo      estado turno                                  
Governador AC     1     Gladson Cameli             0.5316
                        Jorge Viana                0.2267
                        Mara Rocha                 0.1036
                        Petecão                    0.0601
                        Nulo                       0.0463
...                                                   ...
Senador    TO     1     Pastor Claudemir Lopes     0.0235
                        Andrea Schimdt             0.0066
                        Lúcia Viana                0.0019
                        Marcelo Claudio            0.0008
                        Lazara Merley              0.0006

[144 rows x 1 columns]

In [232]:
proporcao_estrato_pop = (
    populacao
    .query("cargo == 'Presidente'")
    .groupby(["cargo", "estado", "turno"])
    ["nome"]
    .apply(lambda s: s.value_counts(normalize=True))
    .to_frame("proporcao")
    .assign(
        nome=lambda df_: df_.index.get_level_values(-1)
    )
    .droplevel(-1)
)
proporcao_estrato_pop

/var/folders/xs/y768pf612tscp94by_62zm040000gn/T/ipykernel_11958/156164345.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["cargo", "estado", "turno"])


proporcao            nome
cargo      estado turno                           
Presidente AC     1         0.6045  Jair Bolsonaro
                  1         0.2830            Lula
                  1         0.0441    Simone Tebet
                  1         0.0270      Ciro Gomes
                  1         0.0226          Branco
...                            ...             ...
           TO     1         0.0001    Léo Péricles
                  2         0.4997            Lula
                  2         0.4724  Jair Bolsonaro
                  2         0.0200            Nulo
                  2         0.0079          Branco

[119 rows x 2 columns]

In [233]:
peso_estrato_pop = (
    sample_sizes
    .query("cargo == 'Presidente'")
    .groupby("turno", as_index=False)
    ["N"].apply(lambda s: s / s.sum())
    .to_frame("Wh")
    .droplevel(0)
    .reorder_levels(["cargo", "estado", "turno"])
)

In [234]:
(
    proporcao_estrato_pop
    .join(peso_estrato_pop)
    .groupby(["nome", "turno"])
    .apply(lambda df_: (df_["proporcao"] * df_["Wh"]).sum())
    .unstack("turno")
)

/var/folders/xs/y768pf612tscp94by_62zm040000gn/T/ipykernel_11958/636204458.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df_: (df_["proporcao"] * df_["Wh"]).sum())


turno,1,2
nome,,
Branco,0.0097,0.0094
Ciro Gomes,0.0234,NaN
Eymael,0.0001,NaN
Felipe D'Avila,0.0018,NaN
Jair Bolsonaro,0.4429,0.4944
Lula,0.4591,0.4747
Léo Péricles,0.0002,NaN
Nulo,0.0154,0.0215
Padre Kelmon,0.0006,NaN
